In [22]:
from IPython.core.interactiveshell import oinspect
from peewee import *

db = SqliteDatabase('Lab3_gai.db')

# Определяем базовую модель, которая определяет нашу БД
class BaseModel(Model):
    class Meta:
        database = db

# Определяем модель авто
class Auto(BaseModel):
    auto_id = PrimaryKeyField(column_name='AutoId',null=False)
    mark = TextField(column_name='Mark', null=False)
    plate = TextField(column_name='Plate', null=False)
    fio = TextField(column_name='Fio', null=False)
    year = IntegerField(column_name='YearOfIssue', null=False)
    capacity = IntegerField(column_name='Capacity', null=False)

    class Meta:
        table_name = 'Auto'
        order_by = ('Auto_id',) 

#определяем таблицу Ведомость
class Taxes(BaseModel):
  tax_id = PrimaryKeyField(column_name='TaxId',null=False)
  tax_fio = ForeignKeyField(Auto, related_name='tax_fio', to_field='fio', on_delete='cascade', on_update='cascade')
  month = TextField(column_name='Month', null=False)
  amount = IntegerField(column_name='Amount', null=False)
    
  class Meta:
        table_name = 'Taxes'
        order_by = ('Tax_id',)

if __name__ == '__main__':
    try:
        db.connect()
        Auto.create_table()
    except peewee.InternalError as px:
        print(str(px))
    try:
        Taxes.create_table()
    except peewee.InternalError as px:
        print(str(px))

In [23]:
def add_auto(mark, plate, fio, year, capacity):
    auto = Auto(
        mark=mark,
        plate=plate,
        fio=fio,
        year=year,
        capacity=capacity
    )
    auto.save()

In [24]:
def add_tax(tax_fio, month, amount):
    fio_exist = True
    try:
        id = Auto.select().where(Auto.fio == tax_fio).get()
    except DoesNotExist as de:
        fio_exist = False
 
    if fio_exist:
        tax = Taxes(
            tax_fio=tax_fio,
            month=month,
            amount=amount,
        )
        tax.save()

In [25]:
def update_owner(id, new_fio):
    new_owner = Auto.get(Auto.auto_id == id)
    new_owner.fio = new_fio
    new_owner.save()

In [26]:
def select_owner(fio):
    return Auto.get(Auto.fio == fio)

In [27]:
def delete_auto(fio):
    auto = Auto.get(Auto.fio == fio)
    auto.delete_instance()

In [28]:
add_auto('BMW', 'X809АВ', 'Иванов H.P.', 2010, 1400)
add_auto('Audi', 'М432АР', 'Антонов И.В.', 2005, 2100)
add_auto('Ford', 'Т889НН', 'Петров О.Д.', 2000, 1750)
add_auto('Honda', 'В139ОВ', 'Денисов И.А.',2004, 1600)
add_auto('KIA', 'Е961ЕО', 'Иванов М.А.',2015, 1520)

In [29]:
add_tax('Иванов H.P.', 'Сентябрь', 30000)
add_tax('Антонов И.В.', 'Февраль', 70000)
add_tax('Петров О.Д.', 'Август', 50000)
add_tax('Денисов И.А.','Июль', 45000)
add_tax('Иванов М.А.','Декабрь', 32000)

In [30]:
update_owner(2, 'Давыдов А.И')

In [31]:
owner = select_owner('Петров О.Д.')
print(f"\nАвтомобиль человека с ФИО {owner.fio}: {owner.mark} - {owner.plate}")


Автомобиль человека с ФИО Петров О.Д.: Ford - Т889НН


In [32]:
delete_auto('Иванов М.А.')

In [33]:
def select_all_autos():
  return Auto.select()

def select_all_taxes():
    return Taxes.select

def auto_join(amount):
  return  Auto.select().join(Taxes,on=(Auto.fio==Taxes.tax_fio)).where(Taxes.amount<amount)

In [34]:
autos = select_all_autos()
auto_data = []
for entry in autos:
    auto_data.append({
        'ID':entry.auto_id,
        'Mark': entry.mark,
        'plate': entry.plate,
        'FIO': entry.fio,
        'Year': entry.year,
        'capacity': entry.capacity
    })
print('\nВсе Записи: ') 
for data in auto_data:
  print(data)


Все Записи: 
{'ID': 1, 'Mark': 'BMW', 'plate': 'X809АВ', 'FIO': 'Иванов H.P.', 'Year': 2010, 'capacity': 1400}
{'ID': 2, 'Mark': 'Audi', 'plate': 'М432АР', 'FIO': 'Давыдов А.И', 'Year': 2005, 'capacity': 2100}
{'ID': 3, 'Mark': 'Ford', 'plate': 'Т889НН', 'FIO': 'Петров О.Д.', 'Year': 2000, 'capacity': 1750}
{'ID': 4, 'Mark': 'Honda', 'plate': 'В139ОВ', 'FIO': 'Денисов И.А.', 'Year': 2004, 'capacity': 1600}


In [35]:
auto_entry=auto_join(100000)
for entry in auto_entry:
    print(f"{entry.fio} - {entry.plate}")

Иванов H.P. - X809АВ
Петров О.Д. - Т889НН
Денисов И.А. - В139ОВ


In [36]:
autos = select_all_autos()
for entry in autos:
    if entry.capacity < 1500:
        print(f"Налог для {entry.fio} - {entry.plate} равен - {3*12}" )
    elif entry.capacity > 1900:
        print(f"Налог для {entry.fio} - {entry.plate} равен - {7*12}" )
    else:
        print(f"Налог для {entry.fio} - {entry.plate} равен - {5*12}" )

Налог для Иванов H.P. - X809АВ равен - 36
Налог для Давыдов А.И - М432АР равен - 84
Налог для Петров О.Д. - Т889НН равен - 60
Налог для Денисов И.А. - В139ОВ равен - 60
